<a href="https://colab.research.google.com/github/popelucha/STEAM/blob/main/STEAM_Nase_jmena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naše jména
Vezmeme databázi českých jmen a příjmení a budeme s nimi dělat různé legrácky.
Data jsme převzali z webů:
* http://www.simandl.asp2.cz/CetnostJmen.aspx
* https://www.kalendar.wz.cz/cetnost.php
Dříve se data vyskytovala na stránkách Ministerstva vnitra, později byla kvůli ochraně osobních údajů stažena.
Data z webů jsme upravili tak, aby se s nimi snadno pracovalo.

In [ ]:
!wget https://nlp.fi.muni.cz/~xpopelk/jmena/jmena_muzska.csv

In [ ]:
!wget https://nlp.fi.muni.cz/~xpopelk/jmena/jmena_zenska.csv

In [ ]:
!wget https://nlp.fi.muni.cz/~xpopelk/jmena/prijmeni_muzska.csv

In [ ]:
import pprint
import pandas as pd

In [ ]:
jmena = pd.read_csv('jmena_muzska.csv', usecols=[1, 2], keep_default_na=False)

In [ ]:
prijmeni = pd.read_csv('prijmeni_muzska.csv', usecols=[1, 2], keep_default_na=False)

## Data
V dalším kroku načteme data o jménech a příjmeních v ČR a počtu jejich výskytů.
Tipni si:
* kolik je v ČR různých mužských jmen?
* kolik je v ČR různých ženských jmen?
* kolik je v ČR různých příjmení? (bohužel, odpověď neznáme, protože naše data obsahují jen příjmení s pěti a více nositeli na území ČR)
* které mužské jméno, ženské jméno a příjmení jsou nejčastější?
* kolikáte v pořadí je tvoje vlastní jméno?

TIP: pro načtení ženských jmen musíš změnit jméno souboru `jmena_muzska.csv` na `jmena_zenska.csv` v buňce nahoře

## Jak to vypadá?
Podíváme se na mužská jména, když ale změníš jméno souboru, dostaneš ženská jména.
Pro ženská příjmení bohužel nemáme data. Dala by se získat z mužských příjmení použitím pravidel pro přechylování.
Vypíšeme obsah proměnných `jmena` a `prijmeni`.

In [ ]:
jmena

In [ ]:
prijmeni

Práce s balíčkem pandas je docela jednoduchá. Pokud pracuješ s Excelem, je to ještě jednodušší. Pandas toho umí víc než Excel. Pandas také zvládne velké soubory dat. To je dobře vidět v předchozím výpisu. Pomocí `loc` a `iloc` lze v tabulce dobře hledat.

In [ ]:
jmena.loc[jmena.name=='Pavel'] # vidíme, že Pavlů je v ČR 207100 a je to třetí nejčastější mužské křestní jméno

## Složitější hledání
### Co se rýmuje?
V proměnné `rymuj` je vzor toho, jak by měla jména vypadat. Můžeš změnit tak, aby jméno končilo stejně jako tvoje.

Speciální znaky:
* `.` cokoli: `D.n` = Dan, Don, Den, Drn, ...
* `*` opakuj předchozí 0 až nekonečně: `An*a` = Ana, Anna, Annna, Aa, ...
* `^` začátek textu: `^Marie` = Marie, Marieta, Marie-Linda, ...
* `$` konec textu: `.*el$` = Karel, Pavel, Daniel, Marcel
* `[` a `]` alternativa - platí jedno z toho, co je v závorce: `Kr[iy]stian` = Kristian, Krystian
* `?` opakuj předchozí 0 nebo 1: `Nikolas?` = Nikola, Nikolas

In [ ]:
rymuj = r'.*dan$'
jmena.loc[jmena.name.str.match(rymuj, case=False)]

### Příjmení, se kterým nevycestuješ
Najdeme příjmení, které obsahuje (nejmíň) čtyři písmena č, ř, š, ž, ť, ď nebo ň.

In [ ]:
hacky = r'.*[čřžšďťň].*[čřžšďťň].*[čřžšďťň].*[čřžšďťň].*'
prijmeni.loc[prijmeni.name.str.match(hacky, case=False)]

## Jména postav do knihy

Jak hledají spisovatelé jména pro svoje postavy?

In [ ]:
jmena.sample().name.values[0] + ' ' + prijmeni.sample().name.values[0]

Ok. Takhle se přece nikdo nejmenuje. Někoho obyčejnějšího tam nemáte?
Jo, máme. Můžeme funkci `sample` dát jako váhy právě četnosti jmen. Výsledkem bude, že Jakub nebo Michal se zvolí s větší pravděpodností než Godfried nebo Jorge.

In [ ]:
jmena.sample(weights=jmena.freq).name.values[0] + ' ' + prijmeni.sample(weights=prijmeni.freq).name.values[0]

### Vymeň první písmena
U svého jména už to asi zkusil každý. Zkusme to u naší literární postavy.

In [ ]:
nahodne_jmeno = jmena.sample(weights=jmena.freq).name.values[0]
nahodne_prijmeni = prijmeni.sample(weights=prijmeni.freq).name.values[0]
vymenene_jmeno = nahodne_prijmeni[0] + nahodne_jmeno[1:] + ' ' + nahodne_jmeno[0] + nahodne_prijmeni[1:]
print('''
Kapitola první

Jmenuji se {nahodne_jmeno} a jsem docela obyčejný člověk.
Chodím do školy. Tak jako většina lidí mého věku. Ale spokojený s tím nejsem.
Všechno začalo tím, že se mi posmívali za jméno.
Na jménu {nahodne_jmeno} není nic špatného, a proto - aby na něm něco špatného bylo -
začali mi vyměňovat první písmena. {vymenene_jmeno}. Ohromně se tím bavili.
  '''.format(nahodne_jmeno = nahodne_jmeno+' '+nahodne_prijmeni, vymenene_jmeno = vymenene_jmeno)
)

## Statistika nuda je
Můžeme se podívat, která jména jsou nejčetnější. Nebo taky nejexotičtější.
Měň hodnoty v `sort_values` a podívej se na tabulku.

In [ ]:
jmena.sort_values(by=['freq'], ascending=False)[:10]

Dají se hledat i jiné podivnosti. Třeba na jaká písmena naše jména nejčastěji končí.

In [ ]:
jmena.groupby(jmena.name.str[-1]).count().sort_values(by='freq', ascending=False)

In [ ]:
jmena.loc[jmena.name.str[-3:]=='mbo'] # jmena koncici na -mbo

In [ ]:
prijmeni.groupby(prijmeni.name.str.len()).count() # prijmeni podle delky

In [ ]:
prijmeni.loc[prijmeni.name.str.len()==3] # tripismenna prijmeni

## Vizualizace četnosti

In [ ]:
ax = jmena.sort_values(by='freq', ascending=False).plot.line()
ax.get_xaxis().set_visible(False)

Křivka, která vypadá jako písmeno L ukazuje rozdělení (rozložení, distribuci) jmen v populaci. Znamená, že velká část lidí má velmi frekventovaná jména a menší část lidí má jména, která jsou velmi vzácná. Klidně jsou jediní nositelé svého jména v celé ČR.

In [ ]:
pocet_jmen = len(jmena)
unikatni_jmena = jmena.loc[jmena.freq==1, 'freq'].count()
podil_unikatnich_jmen = round(unikatni_jmena/pocet_jmen*100,2)
neunikatni_jmena = jmena.loc[jmena.freq>1, 'freq'].count()
soucet_neunikatnich_jmen = jmena.loc[jmena.freq>1, 'freq'].sum()
print("""
V ČR je celkem {} různých mužských jmen.
Z tohoto počtu je {} % jmen ({} lidí), které má jen jeden člověk v celé zemi.
{} jmen má v ČR víc než jednoho nositele.
Celkem je v ČR {} lidí, které mají aspoň jednoho jmenovce.
""".format(pocet_jmen, podil_unikatnich_jmen, unikatni_jmena, neunikatni_jmena, soucet_neunikatnich_jmen))